In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import xarray as xr

In [2]:
# Load data
train_ds = xr.open_dataset("/glade/scratch/lverhoef/gdl_toy_ds/train_data.nc")
validation_ds = xr.open_dataset("/glade/scratch/lverhoef/gdl_toy_ds/validation_data.nc").isel(p=slice(0, 1000))

In [3]:
X_train = tf.expand_dims(tf.convert_to_tensor(train_ds['ellipse']), axis=-1)
Y_train = tf.convert_to_tensor(train_ds[['maj_len', 'min_len']].to_array().transpose())

2022-06-09 12:55:22.638136: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-09 12:55:23.229402: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0


In [4]:
X_val = tf.expand_dims(tf.convert_to_tensor(validation_ds['ellipse']), axis=-1)
Y_val = tf.convert_to_tensor(validation_ds[['maj_len', 'min_len']].to_array().transpose())

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2))

In [6]:
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mse']
)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 64)       0

In [8]:
history = model.fit(
    x=X_train, y=Y_train,
    epochs=10, validation_data=(X_val, Y_val),
    shuffle=True
)

Epoch 1/10


2022-06-09 12:56:01.845446: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


313/313 [==============================] - 6s 10ms/step - loss: 41.1478 - mse: 41.1478 - val_loss: 19.8717 - val_mse: 19.8717
Epoch 2/10
313/313 [==============================] - 3s 9ms/step - loss: 20.4932 - mse: 20.4932 - val_loss: 15.8649 - val_mse: 15.8649
Epoch 3/10
313/313 [==============================] - 3s 9ms/step - loss: 15.9344 - mse: 15.9344 - val_loss: 17.1673 - val_mse: 17.1673
Epoch 4/10
313/313 [==============================] - 3s 9ms/step - loss: 15.0823 - mse: 15.0823 - val_loss: 13.4524 - val_mse: 13.4524
Epoch 5/10
313/313 [==============================] - 3s 9ms/step - loss: 13.5303 - mse: 13.5303 - val_loss: 11.6856 - val_mse: 11.6856
Epoch 6/10
313/313 [==============================] - 3s 9ms/step - loss: 10.1014 - mse: 10.1014 - val_loss: 6.1743 - val_mse: 6.1743
Epoch 7/10
313/313 [==============================] - 3s 9ms/step - loss: 5.1048 - mse: 5.1048 - val_loss: 5.1294 - val_mse: 5.1294
Epoch 8/10
313/313 [==============================] - 3s 9ms/ste

In [9]:
model.predict(X_val[0:10, :, :, :])

1/1 [==============================] - 0s 148ms/step


array([[29.513882 , 16.94258  ],
       [35.19102  , 17.176388 ],
       [37.613842 ,  7.7946606],
       [28.239143 , 13.750446 ],
       [24.119587 ,  4.4526515],
       [27.158287 ,  8.700836 ],
       [11.865602 ,  3.6364121],
       [26.792904 ,  3.041811 ],
       [24.937141 ,  6.723087 ],
       [36.347233 ,  0.9402595]], dtype=float32)